In [42]:
from sympy import *
from sympy.functions.special.spherical_harmonics import Ynm, Ynm_c

r_i, theta_i, phi_i = symbols("r_i theta_i phi_i", real = True)
r_j, theta_j, phi_j = symbols("r_j theta_j phi_j", real = True)


In [43]:
def calc(lp, mp, l, m, k, km):
    return integrate(
        4*pi/(2*k+1)*Ynm_c(k,km, theta_i, phi_i).expand(func=True)*Ynm(k, km, theta_j, phi_j).expand(func=True)*
                (Ynm_c(lp, mp, theta_j, phi_j).expand(func=True)*Ynm(l, m, theta_j, phi_j).expand(func=True)*
                 # this is multiplying Y_lpmp(i), we can just average over the angles after this product
                 Ynm(lp, mp, theta_i, phi_i).expand(func=True)*
                 # to project it into the Y_lpmp for the (l, m) orbital
                 4*pi*Ynm_c(l, m, theta_i, phi_i).expand(func=True)*
                 sin(theta_j)*                                     # jacobian for integration on dummy j
                 sin(theta_i)                                      # jacobian for integration in i, to average on angle
                ).factor().simplify(),
    (theta_j, 0, pi), (phi_j, 0, 2*pi), (theta_i, 0, pi), (phi_i, 0, 2*pi))/4/pi # averaged here


In [44]:
def sumK(lp, mp, l, m, k):
    R = 0
    for km in range(-k, k+1):
        R += (calc(lp, mp, l, m, k, km)).evalf()
    return R


In [49]:
def runVd():
    for k in [0, 2]:
        for l in [0, 1]:
            R = 0
            for m in range(-l, l+1):
                R += (sumK(l, m, l, m, k))
            print("Vd(%d,%d): %.10f" % (l, k, R/(2*l+1)))


In [50]:
runVd()

Vd(0,0): 1.0000000000
Vd(1,0): 1.0000000000
Vd(0,2): 0.0000000000
Vd(1,2): 0.0800000000


In [51]:
def runVex():
    for k in [0, 1, 2]:
        for l1 in [0, 1]:
            for l2 in [0, 1]:
                R = 0
                for m1 in range(-l1, l1+1):
                    for m2 in range(-l2, l2+1):
                        R += (sumK(l1, m1, l2, m2, k))
                print("Vex(%d,%d,%d): %.10f" % (l1, l2, k, R/(2*l1+1)/(2*l2+1)))

In [52]:
runVex()

Vex(0,0,0): 1.0000000000
Vex(0,1,0): 0.0000000000
Vex(1,0,0): 0.0000000000
Vex(1,1,0): 0.3333333333
Vex(0,0,1): 0.0000000000
Vex(0,1,1): 0.3333333333
Vex(1,0,1): 0.3333333333
Vex(1,1,1): 0.0000000000
Vex(0,0,2): 0.0000000000
Vex(0,1,2): 0.0000000000
Vex(1,0,2): 0.0000000000
Vex(1,1,2): 0.1333333333
